In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-12-17 2021-12-20


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|███████████████████▏                                                                                                  | 82/505 [00:13<01:05,  6.42it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 53%|██████████████████████████████████████████████████████████████▎                                                      | 269/505 [00:44<00:44,  5.28it/s]

project_market prices documents must be a non-empty list


 61%|███████████████████████████████████████████████████████████████████████▏                                             | 307/505 [00:51<00:37,  5.25it/s]

...

If using all scalar values, you must pass an index
'NoneType' object does not support item assignment
If using all scalar values, you must pass an index
'NoneType' object does not support item assignment


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 504/505 [01:27<00:00,  7.79it/s]

If using all scalar values, you must pass an index
'NoneType' object does not support item assignment
If using all scalar values, you must pass an index
'NoneType' object does not support item assignment


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [01:28<00:00,  5.73it/s]

If using all scalar values, you must pass an index
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|███████████████▍                                                                                                      | 66/505 [00:07<00:43,  9.98it/s]

'date'


 16%|██████████████████▋                                                                                                   | 80/505 [00:08<00:31, 13.61it/s]

'date'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [01:02<00:00,  8.02it/s]

'date'
'date'
'date'
'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
95,0.164964,0.276230,0.282245,0.170252,0.129324,CERN,Cerner,Health Care
362,-0.403882,-0.386748,0.007522,0.015158,0.079955,PENN,Penn National Gaming,Consumer Discretionary
405,0.023897,-0.215748,0.126653,0.003604,0.052471,RCL,Royal Caribbean Group,Consumer Discretionary
341,-0.143279,-0.281879,0.142777,-0.012627,0.051189,NCLH,Norwegian Cruise Line Holdings,Consumer Discretionary
187,-0.003501,0.124882,0.098955,0.039103,0.049472,FDX,FedEx,Industrials
140,0.552356,0.009856,0.012747,0.010426,0.049078,DXCM,DexCom,Health Care
315,1.638504,-0.135712,-0.050900,0.083863,0.045316,MRNA,Moderna,Health Care
382,0.021746,0.001992,0.109427,-0.014447,0.045274,PTC,PTC,Information Technology
85,-0.103042,-0.299348,0.115995,-0.003272,0.040410,CCL,Carnival Corporation,Consumer Discretionary
48,-0.523132,-0.614728,0.112365,0.021562,0.037575,ANET,Arista Networks,Information Technology


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Health Care,0.208972,0.026381,0.065827,0.014721,0.002603
Communication Services,0.097733,-0.085100,0.046514,0.013187,0.001350
Real Estate,0.481448,0.088842,0.065706,-0.000488,-0.001095
Information Technology,0.236823,0.047945,0.023356,-0.010730,-0.003097
Consumer Discretionary,0.241167,0.020088,0.018236,-0.020669,-0.006092
Materials,0.274495,0.065869,0.048692,0.001549,-0.012255
Utilities,0.156173,0.079896,0.068036,0.002218,-0.014090
Consumer Staples,0.126591,0.071919,0.083538,0.006908,-0.014421
Energy,0.530609,-0.010794,0.006276,-0.022634,-0.016439
